# Project Group - 

Members: 

Student numbers: 

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: Jurian Fijen

**Author 2**: Quirine Japikse

**Author 3**: Christos Paschalidis

**Author 4**: Kristian Terlien

**Author 5**: Timo Locher

# Data Used

# Data Pipeline

# Load Libraries

In [5]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  

# Data Import / Filtering

### NS Data

In [6]:
def read_ns_data(station:str,year:str):
    """
    Input: NS station and year
    Method: Use Selenium which uses local browser to load a webpage
    Reason: NS page is dynamic (not static) therefore the page loads in several phases (Due to animations)
    Output: Statistics for the input station and year

    """
    URL = "https://dashboards.nsjaarverslag.nl/reizigersgedrag/" + station +"?dtYear=" + year
    browser = webdriver.Safari()  
    browser.get(URL)  
    time.sleep(3)
    html_source = browser.page_source  
    browser.quit()
    soup = BeautifulSoup(html_source,'html.parser')  
    results = soup.find(id="content")
    kpi_value_containers = soup.find_all(class_='db-kpi_value-container')

    # Initialize a dictionary to store the extracted values
    kpi_values = {}

    # Iterate through the value containers
    for container in kpi_value_containers:
        # Find the associated title (assuming it's in the parent div)
        title = container.find_previous(class_='db-kpi_title').text.strip()

        # Extract the value (assumes it's within a <span> element)
        value_element = container.find('span', class_='db-kpi_value')
        value = value_element.text.strip()

        # Check if the title is already in the dictionary
        if title in kpi_values:
            # If it is, append the value to the existing list
            kpi_values[title].append(value)
        else:
            # If it's not, create a new list with the value
            kpi_values[title] = [value]
            
    # create data for dataframe
    data ={}
    for title, value in kpi_values.items():
        if len(value) == 2:
            data[title + '_voor'] = value[0]
            data[title + '_na'] = value[1]
        else:
            data[title] = value

    df = pd.DataFrame(data)
    df.index = pd.MultiIndex.from_tuples([(station, year)], names=['Station', 'Year'])
    return df

In [7]:
# test function
read_ns_data('delft','2019')

,,Reizigers per dag,Klantoordeel,In- en uitstappers,Overstappers,Ochtendspits,Avondspits,Daluren,Lopend_voor,Lopend_na,Fiets_voor,Fiets_na,Bus/tram/metro_voor,Bus/tram/metro_na,Auto (bestuurder)_voor,Auto (bestuurder)_na,Auto (passagier)_voor,Auto (passagier)_na,(Deel)taxi_voor,(Deel)taxi_na
Station,Year,,,,,,,,,,,,,,,,,,,
delft,2019,40.818,92%,40.435,383,22%,19%,59%,27%,38%,53%,37%,17%,21%,1%,1%,2%,3%,0%,0%


In [8]:
ns_stations = ['alphen-aan-den-rijn','barendrecht','bodegraven','boskoop','boskoop-snijdelwijk','capelle-schollevaar',
               'de-vink','delft','delft-campus','den-haag-centraal','den-haag-hs','den-haag-laan-van-noi','den-haag-mariahoeve',
               'den-haag-moerwijk','den-haag-ypenburg','dordrecht','dordrecht-zuid','gouda','gouda-goverwelle','hillegom',
               'lansingerland-zoetermeer','leiden-centraal','leiden-lammenschans','nieuwerkerk-a-d-ijssel','rijswijk',
               'rotterdam-alexander','rotterdam-blaak','rotterdam-centraal','rotterdam-lombardijen','rotterdam-noord',
               'rotterdam-zuid','sassenheim','schiedam-centrum','voorburg','voorhout','voorschoten','waddinxveen','waddinxveen-noord',
               'waddinxveen-triangel','zoetermeer','zoetermeer-oost','zwijndrecht'
              ]
years = ['2019','2020','2021','2022']

#### Create dataframe for all stations,years

In [ ]:
# Create Dataframe
"""
Don't run it, just import the csv file!!!
"""

df_ns_data = pd.DataFrame()
# Loop through stations and years
for station in ns_stations:
    for year in years:
        # Call the read_ns_data function
        df = read_ns_data(station, year)
        
        # Concatenate the obtained DataFrame with the final_df
        df_ns_data = pd.concat([df_ns_data, df])
df_ns_data.to_csv('unfiltered_ns_data.csv') # save dataset to csv

In [ ]:
# Filter Dataframe
"""
Don't run it, just import the csv file!!!
"""

df_ns = pd.read_csv('unfiltered_ns_data.csv',dtype={'Reizigers per dag': str,'In- en uitstappers':str,'Overstappers':str}) # load the csv

# Account for passenger values over 1000 which are displayed with a dot
def multiply_by_1000(value):
    if '.' in value:
        return int( float(value) * 1000 )
    else:
        return value
# 'In- en uitstappers','Overstappers'
df_ns['Reizigers per dag'] = df_ns['Reizigers per dag'].apply(multiply_by_1000)
df_ns['In- en uitstappers'] = df_ns['In- en uitstappers'].apply(multiply_by_1000)
df_ns['Overstappers'] = df_ns['Overstappers'].apply(multiply_by_1000)

#df_ns.to_csv('df_ns.csv') #save dataframe


# df_ns dataframe is now ready to use

In [9]:
df_ns = pd.read_csv('df_ns.csv') # Run this instead

#### Dataframe is now ready to use

In [10]:
# extract data for a specific station on a specific year
df_ns[(df_ns['Station'] == 'alphen-aan-den-rijn') & (df_ns['Year'] == 2019)]

,Unnamed: 0,Station,Year,Reizigers per dag,Klantoordeel,In- en uitstappers,Overstappers,Ochtendspits,Avondspits,Daluren,...,Fiets_voor,Fiets_na,Bus/tram/metro_voor,Bus/tram/metro_na,Auto (bestuurder)_voor,Auto (bestuurder)_na,Auto (passagier)_voor,Auto (passagier)_na,(Deel)taxi_voor,(Deel)taxi_na
0,0,alphen-aan-den-rijn,2019,12630,72%,10996,1634,32%,13%,55%,...,48%,24%,17%,17%,8%,2%,4%,11%,0%,0%


In [11]:
# extract data for a specific year
df_ns[df_ns['Year'] == 2019].head()

,Unnamed: 0,Station,Year,Reizigers per dag,Klantoordeel,In- en uitstappers,Overstappers,Ochtendspits,Avondspits,Daluren,...,Fiets_voor,Fiets_na,Bus/tram/metro_voor,Bus/tram/metro_na,Auto (bestuurder)_voor,Auto (bestuurder)_na,Auto (passagier)_voor,Auto (passagier)_na,(Deel)taxi_voor,(Deel)taxi_na
0,0,alphen-aan-den-rijn,2019,12630,72%,10996,1634,32%,13%,55%,...,48%,24%,17%,17%,8%,2%,4%,11%,0%,0%
4,4,barendrecht,2019,6066,79%,6066,0,33%,12%,55%,...,34%,22%,4%,8%,28%,6%,13%,24%,0%,0%
8,8,bodegraven,2019,3448,68%,3448,0,37%,10%,53%,...,42%,25%,1%,2%,16%,4%,5%,18%,0%,0%
12,12,boskoop,2019,1432,78%,1432,0,33%,9%,58%,...,58%,27%,0%,7%,6%,3%,6%,9%,1%,0%
16,16,boskoop-snijdelwijk,2019,667,92%,667,0,35%,7%,58%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
